In [ ]:
# Install condacolab
!pip install -q condacolab
import condacolab
condacolab.install()

!conda create -n internvl python=3.9 -y

!source activate internvl

In [ ]:
!git clone https://github.com/Czi24/InternVL-20240714.git
!pip install torch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cu118

!pip install flash-attn==2.3.6 --no-build-isolation
!pip install transformers==4.37.2
!pip install opencv-python termcolor yacs pyyaml scipy
!pip install deepspeed==0.13.5
!pip install pycocoevalcap tqdm

%cd /content/InternVL-20240714/internvl_chat_llava/
!pip install -e .

In [2]:
!pip install huggingface_hub protobuf
from huggingface_hub import snapshot_download
%cd /content
snapshot_download(repo_id="OpenGVLab/InternVL2-1B", local_dir="InternVL2-1B")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.3/309.3 kB 20.2 MB/s eta 0:00:00
/content


/usr/local/lib/python3.10/site-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 18 files:   0%|          | 0/18 [00:00<?, ?it/s]

examples/image1.jpg:   0%|          | 0.00/78.1k [00:00<?, ?B/s]

conversation.py:   0%|          | 0.00/15.1k [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.02k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.38M [00:00<?, ?B/s]

'/content/InternVL2-1B'

In [3]:
%cd /content/InternVL-20240714/internvl_chat_llava/
import threading
import subprocess

# start Controller
threading.Thread(
    target=lambda: subprocess.run(
        ['python3', '-m', 'llava.serve.controller', '--host', '0.0.0.0', '--port', '10000'],
        check=True
    ),
    daemon=True
).start()

/content/InternVL-20240714/internvl_chat_llava


In [4]:
%cd /content/InternVL-20240714/internvl_chat
import threading
import subprocess

# Define the command to run the model worker
command = [
    'python3', '-m', 'internvl.serve.model_worker',
    '--host', '0.0.0.0',
    '--controller', 'http://localhost:10000',
    '--port', '40001',
    '--worker', 'http://localhost:40001',
    '--model-path', '/content/InternVL2-1B'
]

# Run the command in a separate thread
thread = threading.Thread(
    target=lambda: subprocess.run(command, check=True, shell=False),
    daemon=True
)
thread.start()


/content/InternVL-20240714/internvl_chat


In [5]:
%cd /content/InternVL-20240714/internvl_chat_llava
!python3 -m llava.serve.gradio_web_server --controller http://localhost:10000 --model-list-mode reload --share

/content/InternVL-20240714/internvl_chat_llava
2024-07-15 12:26:18 | INFO | gradio_web_server | args: Namespace(host='0.0.0.0', port=None, controller_url='http://localhost:10000', concurrency_count=10, model_list_mode='reload', share=True, moderate=False, embed=False)
2024-07-15 12:26:18 | INFO | gradio_web_server | Models: ['InternVL2-1B']
2024-07-15 12:26:18 | INFO | gradio_web_server | Namespace(host='0.0.0.0', port=None, controller_url='http://localhost:10000', concurrency_count=10, model_list_mode='reload', share=True, moderate=False, embed=False)
2024-07-15 12:26:18 | INFO | stdout | Running on local URL:  http://0.0.0.0:7860
2024-07-15 12:26:20 | INFO | stdout | Running on public URL: https://9d789922220b52b26d.gradio.live
2024-07-15 12:26:20 | INFO | stdout | 
2024-07-15 12:26:20 | INFO | stdout | This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
2024-07-15 12:2